In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import plotly.io as pio

In [11]:
def map_tech_to_mode(tech):
    class_mapping = {
    'T_HDV_AJ': 'Air jet',
    'T_HDV_B': 'Bus',
    'T_HDV_R': 'Rail',
    'T_HDV_T': 'HD Truck',
    'T_HDV_W': 'Marine vessel',
    'T_MDV_T': 'MD Truck',
    'T_LDV_C_': 'LD Car',
    'T_LDV_LT': 'LD Truck',
    'T_LDV_M': 'Motorcycle',
    'T_IMP_': 'Fuel use',
    'H2_COMP_100_700': 'Fuel use'
    }

    for prefix, class_name in class_mapping.items():
        if tech.startswith(prefix):
            return class_name
    return 'Other'

def map_tech_to_fuel(tech):
    carrier_mapping = {
        'BEV': 'Battery electric',
        'GSL': 'Gasoline',
        'DSL': 'Diesel',
        'CNG': 'Compressed NG',
        'LNG': 'Liquified NG',
        'JTF': 'Jet Fuel',
        'SPK': 'Synth. Jet Fuel',
        'HFO': 'Heavy Fuel Oil',
        'MDO': 'Marine Diesel Oil',
        'ELC': 'Electricity',
        'ETH': 'Ethanol',	
        'RDSL': 'Ren. Diesel',
    }
    # Order matters 
    if 'PHEV35' in tech:
        return 'PHEV (35-mile AER)'
    if 'PHEV50' in tech:
        return 'PHEV (50-mile AER)'
    if 'PHEV' in tech:
        return 'Plug-in hybrid'
    if 'BEV150' in tech:
        return 'BEV (150-mile AER)'
    if 'BEV200' in tech:
        return 'BEV (200-mile AER)'
    if 'BEV300' in tech:
        return 'BEV (300-mile AER)'
    if 'BEV400' in tech:
        return 'BEV (400-mile AER)'
    if 'FC' in tech:
        return 'Fuel-cell electric'
    if 'HEV' in tech:
        return 'Hybrid'
    if 'H2' in tech:
        return 'Hydrogen'
    if 'BEV_CHRG' in tech:
        return 'LD BEV charger'
    if 'CHRG' in tech:
        return 'Other charger'
    for prefix, carrier in carrier_mapping.items():
        if prefix in tech:
            return carrier 
    return 'Other'

In [12]:
def import_db_capacity_data(db_variant='baseline_life_7', scenario='baseline_life_7', path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    db_file = path + f'canoe_on_12d_{db_variant}.sqlite'
    conn = sqlite3.connect(db_file)

    # filter db tables
    query_net_capacity = f"SELECT * FROM OutputNetCapacity WHERE sector = 'Transport' AND scenario = '{scenario}'"
    query_built_capacity = f"SELECT * FROM OutputBuiltCapacity WHERE sector = 'Transport' AND scenario = '{scenario}'"
    query_existing_capacity = "SELECT * FROM ExistingCapacity WHERE tech like 'T_%'"

    net_cap = pd.read_sql_query(query_net_capacity, conn).drop(columns=['region', 'sector'])
    new_cap = pd.read_sql_query(query_built_capacity, conn).drop(columns=['region', 'sector'])
    ex_cap =  pd.read_sql_query(query_existing_capacity, conn)[['tech', 'vintage', 'capacity', 'units']]
    conn.close()

    # label mode fuel and lifetime classes
    net_cap['mode'] = net_cap['tech'].apply(map_tech_to_mode)
    new_cap['mode'] = new_cap['tech'].apply(map_tech_to_mode)
    ex_cap['mode'] = ex_cap['tech'].apply(map_tech_to_mode)

    net_cap['fuel'] = net_cap['tech'].apply(map_tech_to_fuel)
    new_cap['fuel'] = new_cap['tech'].apply(map_tech_to_fuel)
    ex_cap['fuel'] = ex_cap['tech'].apply(map_tech_to_fuel)

    # extract the last two digits (if present), else NaN - fill NaN with 50 to denote median parent class
    percentile = net_cap['tech'].str.extract(r'_S(\d{2})$')[0]
    net_cap['life'] = 'Survival P' + percentile.fillna('50')
    percentile = new_cap['tech'].str.extract(r'_S(\d{2})$')[0]
    new_cap['life'] = 'Survival P' + percentile.fillna('50')
    percentile = ex_cap['tech'].str.extract(r'_S(\d{2})$')[0]
    ex_cap['life'] = 'Survival P' + percentile.fillna('50')

    # prepare capacity dfs
    net_cap_group = net_cap.groupby(['mode', 'fuel', 'life', 'period'], as_index=False).sum('capacity').drop(columns='vintage').rename(columns={'period': 'vintage'})
    ex_cap_group = ex_cap.copy()
    ex_cap_group['vintage'] = 2021
    ex_cap_group = ex_cap_group.groupby(['mode', 'fuel', 'life', 'vintage', 'units'], as_index=False).sum('capacity')
    new_cap_group = new_cap.groupby(['mode', 'fuel', 'life', 'vintage'], as_index=False).sum('capacity')

    # brand capacity types and merge, filling empty values with 0
    net_cap_group = net_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'net capacity'})
    new_cap_group = new_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'new capacity'})
    ex_cap_group = ex_cap_group[['mode', 'fuel', 'life', 'vintage', 'capacity']].rename(columns={'capacity': 'ex capacity'})
    merged_cap = net_cap_group.merge(new_cap_group, on=['mode', 'fuel', 'life', 'vintage'], how='left').merge(ex_cap_group, on=['mode', 'fuel', 'life', 'vintage'], how='left').fillna(0.)

    # sort and calculate retired capacity
    merged_cap = merged_cap.sort_values(by=['mode', 'fuel', 'life', 'vintage'])
    merged_cap['retired cap'] = 0.

    for _, group in merged_cap.groupby(['mode', 'fuel', 'life']):
        for i in range(1, len(group)):
            current_idx = group.index[i]
            previous_idx = group.index[i-1]

            # calculate retired capacity as netcap_i + newcap_i - netcap_{i-1}, where excap_i = netcap_{i-1}
            merged_cap.loc[current_idx, 'retired cap'] = (
                merged_cap.loc[previous_idx, 'net capacity'] + 
                merged_cap.loc[current_idx, 'new capacity'] - 
                merged_cap.loc[current_idx, 'net capacity']
            )

    for _, group in merged_cap.groupby(['mode', 'fuel', 'life']):
        for i in range(1, len(group)):
            current_idx = group.index[i]
            previous_idx = group.index[i-1]

            # fill the "ex capacity" for years after 2021 using the "net capacity" from the previous year
            merged_cap.loc[current_idx, 'ex capacity'] = merged_cap.loc[previous_idx, 'net capacity']

    merged_cap['retired cap'] = -merged_cap['retired cap']  # negative values for retired capacity
    merged_cap = merged_cap.round(2)
    return merged_cap

In [13]:
import_db_capacity_data()

,mode,fuel,life,vintage,net capacity,new capacity,ex capacity,retired cap
0,Air jet,Other,Survival P50,2021,70.23,1.39,67.90,-0.00
1,Air jet,Other,Survival P50,2021,70.23,1.39,70.23,-1.39
2,Air jet,Other,Survival P50,2025,80.89,10.66,70.23,0.00
3,Air jet,Other,Survival P50,2030,89.21,19.57,80.89,-11.25
4,Air jet,Other,Survival P50,2035,97.07,18.38,89.21,-10.52
...,...,...,...,...,...,...,...,...
1440,Rail,Liquified NG,Survival P50,2030,73.18,33.76,39.42,-0.00
1441,Rail,Liquified NG,Survival P50,2035,80.64,7.45,73.18,-0.00
1442,Rail,Liquified NG,Survival P50,2040,121.54,40.91,80.64,-0.00
1443,Rail,Liquified NG,Survival P50,2045,129.92,8.37,121.54,-0.00


In [14]:
color_fuel_map = {
    'Gasoline': 'red',
    'Diesel': 'brown',
    'Compressed NG': 'tomato',
    'Hybrid': 'darkorange',
    'PHEV (35-mile AER)': 'dodgerblue',
    'PHEV (50-mile AER)': 'darkblue',
    'BEV (150-mile AER)': 'limegreen',
    'BEV (200-mile AER)': 'seagreen',
    'BEV (300-mile AER)': 'olive',
    'BEV (400-mile AER)': 'darkgreen',
    'Plug-in hybrid': 'blue',
    'Battery electric': 'green',
    'Fuel-cell electric': 'mediumvioletred'
}

In [15]:
def color_fuel_map_rgba(alpha=0.4):
    return {
        'Gasoline':          f'rgba(255,   0,   0,   {alpha})',    # red
        'Diesel':            f'rgba(165,  42,  42,  {alpha})',    # brown
        'Compressed NG':     f'rgba(255,  99,  71,  {alpha})',    # tomato
        'Hybrid':            f'rgba(255, 140,   0,  {alpha})',    # darkorange
        'PHEV (35-mile AER)': f'rgba(30, 144, 255,  {alpha})',   # dodgerblue
        'PHEV (50-mile AER)': f'rgba(0,    0,   139, {alpha})',   # darkblue
        'BEV (150-mile AER)': f'rgba(50,  205,  50,  {alpha})',   # limegreen
        'BEV (200-mile AER)': f'rgba(46,  139,  87,  {alpha})',   # seagreen
        'BEV (300-mile AER)': f'rgba(128, 128,   0,  {alpha})',   # olive
        'BEV (400-mile AER)': f'rgba(0,   100,   0,  {alpha})',   # darkgreen
        'Plug-in hybrid':    f'rgba(0,    0,   255,  {alpha})',   # blue
        'Battery electric':  f'rgba(0,   128,   0,   {alpha})',   # green
        'Fuel-cell electric':f'rgba(199,  21,  133,  {alpha})'    # mediumvioletred
    }

In [16]:
def plot_capacity_flow(db_variant='lifetimes', scenario='trn_lifetimes', lifetime_classes=None, scenario_title=None,
                        modes=['LD Car', 'LD Truck', 'MD Truck', 'HD Truck'], color_map=color_fuel_map):
    df = import_db_capacity_data(db_variant=db_variant, scenario=scenario)
    df = df.melt(id_vars=['mode', 'fuel', 'life', 'vintage'],
                       value_vars=['net capacity', 'ex capacity', 'new capacity', 'retired cap'],
                       var_name='cap type', value_name='capacity')

    # df['capacity'] = df['capacity'] / 1E3     # convert to million units
    df['vintage'] = df['vintage'].astype('str')
    df_filtered = df[(abs(df['capacity']) > 1e-3) & (df['mode'].isin(modes)) & (df['cap type'] != 'ex capacity')].reset_index(drop=True)
    if lifetime_classes is None:
        df_filtered = df_filtered.groupby(['mode', 'fuel', 'vintage', 'cap type'], as_index=False).sum('capacity')
    else:
        df_filtered = df_filtered.groupby(['mode', 'life', 'vintage', 'cap type'], as_index=False).sum('capacity')

    
    fig = px.bar(df_filtered, x='cap type', y='capacity', color='fuel' if lifetime_classes is None else 'life', 
            pattern_shape='cap type', pattern_shape_sequence=["", ".", "/"],
            facet_col='vintage', facet_col_spacing=2E-2, 
            facet_row='mode', facet_row_spacing=1.5E-2,
            category_orders={"cap type": ["net capacity", "new capacity", "retired cap"],
                             'vintage': sorted(df['vintage'].unique()),
                             "fuel": list(color_map.keys()) if lifetime_classes is None else [],
                             'life': sorted(df['life'].unique()) if lifetime_classes is not None else [],
                             'mode': modes},
            template='plotly_white', orientation='v', 
            color_discrete_map = color_map if lifetime_classes is None else {},
            color_discrete_sequence=px.colors.qualitative.Bold if lifetime_classes is not None else [],
            text_auto='.2s', width=1200, height=900
            )

    fig.update_layout(
        margin=dict(
            t=65, b=30),
        title=dict(
            text=f'<b>Vehicle fleet stock and flow in ON by vehicle class ({scenario_title})</b>',
            x=0.5, y=0.98, xanchor='center', yanchor='top'),
        yaxis_title_standoff=0,
        legend_title=dict(
            text='<b>Fuel/powertrain type</b>' if lifetime_classes is None else '<b>Lifetime class</b>', 
            font=dict(size=16)),
        bargap=0.1,
        legend=dict(
            traceorder='grouped', orientation='v', yanchor='top', y=0.8, xanchor='center', x=1.15),
        font=dict(
            size=15)
        )

    fig.for_each_trace(lambda trace: trace.update(textfont=dict(size=11)))
    fig.for_each_xaxis(lambda axis: axis.update(title_text='', showticklabels=False))
    fig.for_each_yaxis(lambda axis: axis.update(title_text=''))

    unique_xdomains = sorted(set(fig.layout[axis].domain[0] for axis in fig.layout if axis.startswith('yaxis')))
    n_vintages = df['vintage'].nunique()
    for row_i, _ in enumerate(unique_xdomains, start=1):
        anchor_i = (row_i - 1) * n_vintages + 1   # left-most col in that row
        match_anchor = 'y' if row_i == 1 else f'y{anchor_i}'
        fig.update_yaxes(matches=match_anchor, row=row_i, col=None, nticks=8, zeroline=True, zerolinecolor='black', tickformat='~s')
    
    for annotation in fig.layout.annotations:           # Fix facet cols and facet row annotations
        if 'mode' in annotation.text:
            annotation.text = f"<b>{annotation.text.split('=')[1]}</b>"
            annotation.font.size = 16
            annotation.x = 1.01
            annotation.xanchor = 'center'
        else:
            annotation.text = f"<b>{annotation.text.split('=')[1]}</b>"
            annotation.font.size = 16
            annotation.y = 0
            annotation.yanchor = 'top' 

    shown_legends = set()
    for trace in fig.data:                              # Show only one legend for each fuel type
        trace.name = trace.name.split(",")[0]
        if trace.name not in shown_legends:
            trace.showlegend = True
            shown_legends.add(trace.name)
        else:
            trace.showlegend = False
    
    # enforce fuel/life order in legend
    if lifetime_classes is None:
        _order = list(color_map.keys())
    else: 
        _order = sorted(df['life'].unique())
    fig.data = tuple(
        sorted(fig.data, key=lambda t: (_order.index(t.name) if t.name in _order else len(_order)))
    )

    # add capacity type legends separated by a blank entry
    fig.add_trace(go.Bar(
        x=[None], y=[None],
        showlegend=True,
        legendgroup='Blank',
        legendgrouptitle=None,
        name='',
        marker_color='rgba(0,0,0,0)'
    ))

    for i, (name, pat) in enumerate([
        ("Net capacity",   ""),
        ("New capacity",   "."),
        ("Retired capacity", "/"),
    ]):
        fig.add_trace(go.Bar(
            x=[None], y=[None],
            name=name,
            showlegend=True,
            legendgroup='Capacity type',
            legendgrouptitle=dict(text="<b>Capacity type</b>", font=dict(size=16)) if i == 0 else None,
            marker=dict(
                color='rgba(0,0,0,0)',
                line=dict(color='black', width=1),
                pattern_shape=pat
            )
        ))

    fig.add_annotation(
        text='<b>Fleet capacity (k vehicles)</b>',
        x=-0.08, y=0.5, xref="paper", yref="paper", showarrow=False, textangle=-90, font=dict(size=17))
        
    fig.show()

In [17]:
def plot_capacity_flow(db_variant='baseline_life_7', scenario='baseline_life_7', lifetime_classes=None, scenario_title=None,
                        modes=['LD Car', 'LD Truck', 'MD Truck', 'HD Truck'],
                        cap_types=['net capacity', 'new capacity', 'retired cap'],
                        color_map=color_fuel_map):
    df = import_db_capacity_data(db_variant=db_variant, scenario=scenario)
    df = df.melt(id_vars=['mode', 'fuel', 'life', 'vintage'],
                       value_vars=['net capacity', 'ex capacity', 'new capacity', 'retired cap'],
                       var_name='cap type', value_name='capacity')

    df['capacity'] = df['capacity'] * 1E3     # convert to normal units
    df['vintage'] = df['vintage'].astype('str')
    df_filtered = df[(abs(df['capacity']) > 1e-3) & (df['mode'].isin(modes)) & (df['cap type'].isin(cap_types))].reset_index(drop=True)
    df_filtered = df_filtered[df_filtered['vintage'] != '2021'].copy()
    if lifetime_classes is None:
        df_filtered = df_filtered.groupby(['mode', 'fuel', 'vintage', 'cap type'], as_index=False).sum('capacity')
    else:
        df_filtered = df_filtered.groupby(['mode', 'life', 'vintage', 'cap type'], as_index=False).sum('capacity')

    
    fig = px.bar(df_filtered, x='cap type', y='capacity', color='fuel' if lifetime_classes is None else 'life', 
            pattern_shape='cap type', 
            pattern_shape_sequence=["", ".", "/"] if len(cap_types) == 3 else ["", "/"],
            facet_col='vintage', facet_col_spacing=2E-2, 
            facet_row='mode', facet_row_spacing=0.04,
            category_orders={"cap type": cap_types,
                             'vintage': sorted(df_filtered['vintage'].unique()),
                             "fuel": list(color_map.keys()) if lifetime_classes is None else [],
                             'life': sorted(df_filtered['life'].unique()) if lifetime_classes is not None else [],
                             'mode': modes},
            template='plotly_white', orientation='v', 
            color_discrete_map = color_map if lifetime_classes is None else {},
            color_discrete_sequence=px.colors.qualitative.Bold if lifetime_classes is not None else [],
            text_auto='.2s', width=1200, height=450
            )

    fig.update_layout(
        margin=dict(
            t=65, b=30),
        title=dict(
            text=f'<b>Vehicle fleet stock and flow in ON by vehicle class ({scenario_title})</b>',
            x=0.08, y=0.94, xanchor='left', yanchor='top'),
        yaxis_title_standoff=0,
        legend_title=dict(
            text='<b>Fuel/powertrain type</b>' if lifetime_classes is None else '<b>Lifetime class</b>', 
            font=dict(size=16)),
        bargap=0.1,
        legend=dict(
            traceorder='grouped', orientation='v', yanchor='top', y=1.0, xanchor='center', x=1.15),
        font=dict(
            size=15)
        )

    fig.for_each_trace(lambda trace: trace.update(textfont=dict(size=11)))
    fig.for_each_xaxis(lambda axis: axis.update(title_text='', showticklabels=False))
    fig.for_each_yaxis(lambda axis: axis.update(title_text=''))

    unique_xdomains = sorted(set(fig.layout[axis].domain[0] for axis in fig.layout if axis.startswith('yaxis')))
    n_vintages = df['vintage'].nunique()
    for row_i, _ in enumerate(unique_xdomains, start=1):
        anchor_i = (row_i - 1) * n_vintages + 1   # left-most col in that row
        match_anchor = 'y' if row_i == 1 else f'y{anchor_i}'
        fig.update_yaxes(matches=match_anchor, row=row_i, col=None, nticks=8, zeroline=True, zerolinecolor='black', tickformat='~s')
    
    for annotation in fig.layout.annotations:           # Fix facet cols and facet row annotations
        if 'mode' in annotation.text:
            annotation.text = f"<b>{annotation.text.split('=')[1]}</b>"
            annotation.font.size = 16
            annotation.x = 1.01
            annotation.xanchor = 'center'
        else:
            annotation.text = f"<b>{annotation.text.split('=')[1]}</b>"
            annotation.font.size = 16
            annotation.y = 0
            annotation.yanchor = 'top' 

    shown_legends = set()
    for trace in fig.data:                              # Show only one legend for each fuel type
        trace.name = trace.name.split(",")[0]
        if trace.name not in shown_legends:
            trace.showlegend = True
            shown_legends.add(trace.name)
        else:
            trace.showlegend = False
    
    # enforce fuel/life order in legend
    if lifetime_classes is None:
        _order = list(color_map.keys())
    else: 
        _order = sorted(df['life'].unique())
    fig.data = tuple(
        sorted(fig.data, key=lambda t: (_order.index(t.name) if t.name in _order else len(_order)))
    )

    # add capacity type legends separated by a blank entry
    fig.add_trace(go.Bar(
        x=[None], y=[None],
        showlegend=True,
        legendgroup='Blank',
        legendgrouptitle=None,
        name='',
        marker_color='rgba(0,0,0,0)'
    ))

    for i, (name, pat) in enumerate([
        # ("Net capacity",   ""),
        ("New capacity",   ""),
        ("Retired capacity", "/"),
    ]):
        fig.add_trace(go.Bar(
            x=[None], y=[None],
            name=name,
            showlegend=True,
            legendgroup='Capacity type',
            legendgrouptitle=dict(text="<b>Capacity type</b>", font=dict(size=16)) if i == 0 else None,
            marker=dict(
                color='rgba(0,0,0,0)',
                line=dict(color='black', width=1),
                pattern_shape=pat
            )
        ))

    fig.add_annotation(
        text='<b>Fleet capacity (vehicle units)</b>',
        x=-0.08, y=0.5, xref="paper", yref="paper", showarrow=False, textangle=-90, font=dict(size=17))

    fig.show()
    # fig.write_image("vanilla4.svg", scale=1, engine='kaleido')

In [18]:
fig = plot_capacity_flow(db_variant='vanilla4', 
                   scenario='vanilla4',
                   modes=['LD Truck', 'HD Truck'],
                   cap_types=['new capacity', 'retired cap'], 
                #    lifetime_classes='yes',
                   scenario_title='vanilla model')

In [19]:
plot_capacity_flow(db_variant='baseline_life_3', 
                   scenario='baseline_life_3',
                   modes=['LD Truck', 'HD Truck'],
                   cap_types=['new capacity', 'retired cap'], 
                #    lifetime_classes='yes',
                   scenario_title='3 lifetime classes')

In [20]:
plot_capacity_flow(db_variant='baseline_life_7', 
                   scenario='baseline_life_7',
                   modes=['LD Truck', 'HD Truck'],
                   cap_types=['new capacity', 'retired cap'], 
                #    lifetime_classes='yes',
                   scenario_title='7 lifetime classes')

In [21]:
plot_capacity_flow(db_variant='medgrowth_life_7', 
                   scenario='medgrowth_life_7',
                   modes=['LD Truck', 'HD Truck'],
                   cap_types=['new capacity', 'retired cap'], 
                #    lifetime_classes='yes',
                   scenario_title='Med. Growth (7 lifetimes)')

In [22]:
def load_capacity_flow_scenarios(
        scenarios= {
        # db variant : scenario
        'vanilla4' : 'vanilla4',
        'baseline' : 'baseline',
        "lowgrowth" : "lowgrowth",
        "medgrowth"   : "medgrowth",
        "evgrowth" : "evgrowth",
        "highgrowth": "highgrowth"
        },
        modes=['LD Car', 'LD Truck', 'MD Truck', 'HD Truck']) -> pd.DataFrame:
    dfs = []
    for db_variant, scenario in scenarios.items():
        df = import_db_capacity_data(db_variant, scenario)
        df = df[df['mode'].isin(modes)].copy()  # filter for modes
        df = df.drop(columns=['ex capacity'])   # filter out ex capacity
        df = df[df['vintage'] != 2021].copy()   # filter out 2021 vintage
        df['net capacity'], df['new capacity'], df['retired cap'] = df['net capacity'] * 1E3, df['new capacity'] * 1E3, df['retired cap'] * 1E3     # convert to normal units
        df["scenario"] = scenario
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [23]:
def plot_capacity_flow_multicat_area(
        plot_title='Vehicle fleet stock in Ontario by vehicle class, powertrain and scenario',
        capacity_type='net capacity',
        scenarios={
            # db variant : scenario
            'vanilla4' : 'vanilla4',
            'baseline' : 'baseline',
            "lowgrowth" : "lowgrowth",
            "medgrowth"   : "medgrowth",
            "evgrowth" : "evgrowth",
            "highgrowth": "highgrowth"
        },
        scenario_labels={
            'vanilla4' : 'Vanilla',
            'baseline'  : 'Baseline',
            'lowgrowth'  : 'Low growth',
            'medgrowth'   : 'Med. growth',
            "evgrowth" : "Norway EVs",
            'highgrowth' : 'High growth',
        },
        modes_order=('LD Car', 'LD Truck', 'MD Truck', 'HD Truck'),
        color_map=color_fuel_map
    ):

    df = load_capacity_flow_scenarios(scenarios=scenarios, modes=modes_order)

    # 2) Build the list of bold subplot_titles:
    n_rows = len(modes_order)
    n_cols = len(tuple(scenarios.values()))
    subplot_titles = []
    for r, mode in enumerate(modes_order, start=1):
        for c, scen in enumerate(tuple(scenarios.values()), start=1):
            if r == 1:
                # Top row shows the scenario label (bolded here)
                subplot_titles.append(f'<b>{scenario_labels.get(scen, scen)}</b>')
            else:
                subplot_titles.append("")

    # 3) Create the figure with shared axes (as per your revision):
    fig = make_subplots(
        rows=n_rows, cols=n_cols,
        shared_xaxes=True,
        shared_yaxes=True,
        horizontal_spacing=0.02,
        vertical_spacing=0.03,
        subplot_titles=subplot_titles
    )

    # 4) Loop over each (mode, scenario), pivot, then add stacked‐area traces:
    for r_idx, mode in enumerate(modes_order, start=1):
        for c_idx, scen in enumerate(tuple(scenarios.values()), start=1):
            df_cell = df[(df['mode'] == mode) & (df['scenario'] == scen)].copy()
            if df_cell.empty:
                continue

            # Pivot so index = vintage, columns = fuel, values = capacity_type
            pivot = (
                df_cell
                .groupby(['vintage', 'fuel'])[capacity_type]
                .sum()
                .unstack(fill_value=0)
            )
            # Reindex to force all fuels in the desired order (even if zero):
            pivot = pivot.reindex(columns=list(color_map.keys()), fill_value=0)
            pivot = pivot.sort_index()
            vintages = pivot.index.astype(str).tolist()

            # Now add one stacked‐area trace per fuel:
            # Use stackgroup="one" so that each subplot stacks internally.
            for i, fuel in enumerate(color_map.keys()):
                yvals = pivot[fuel].values.tolist()
                fill_mode = 'tozeroy' if i == 0 else 'tonexty'

                fig.add_trace(
                    go.Scatter(
                        x=vintages,
                        y=yvals,
                        mode='none',    # other options: 'lines', 'markers', 'lines+markers'
                        name=fuel,
                        stackgroup="one",
                        fill=fill_mode,
                        fillcolor=color_map[fuel],       # <-- exact color here
                        line=dict(color=color_map[fuel], width=0),  # no border line
                        legendgroup=fuel,
                        showlegend=(r_idx == 1 and c_idx == 1),  # legend only once
                        hoverinfo='x+y+name'
                    ),
                    row=r_idx, col=c_idx
                )

    # 5) Tweak the x‐axis and y‐axis for this subplot only:
    fig.update_yaxes(title_text='Fleet size (vehicles units)' if capacity_type == 'net capacity' else 'Capacity flow (vehicles units)',
                     row=2, col=1)
    
    fig.update_yaxes(nticks=5, tickformat='~s')
    fig.update_yaxes(tickformat='~s',
                    #  dtick=2500 * 1E3,
                     row=2, col=1)
    
    fig.for_each_xaxis(lambda axis: axis.update(title_text='', showticklabels=False))
    fig.update_xaxes(
        showticklabels=True,
        tickangle=45,
        # ticklabelstandoff=2,
        row=4, col=1,
    )

    fig.update_layout(
        width=250 * n_cols + 200, height=200 * n_rows + 150,
        margin=dict(
            t=70, b=10),
        title=dict(
            text=f'<b>{plot_title}</b>',
            x=0.5, y=0.985, xanchor='center', yanchor='top'),
        yaxis_title_standoff=1,
        legend_title=dict(
            text='<b>Fuel/powertrain type</b>', font=dict(size=16)),
        barmode='stack', bargap=0.15,
        legend=dict(
            orientation='v', yanchor='top', y=0.8, xanchor='center', x=1.23, traceorder='normal'),
        font=dict(size=15),
        template='plotly_white'
        )

    # 8) Finally, add a bold annotation on the right of each row to label the mode:
    for r, mode in enumerate(modes_order, start=1):
        fig.add_annotation(
            text=f'<b>{mode}</b>',
            x=1.005, xref='paper',
            y=1 - (r - 0.5) / 4, yref='paper',
            showarrow=False,
            textangle=90,
            xanchor='left', yanchor='middle',
            font=dict(size=16)
        )

    fig.show()


In [24]:
plot_capacity_flow_multicat_area(
    plot_title='Vehicle new capacity in Ontario by vehicle class, powertrain and scenario',
    capacity_type='new capacity',
    scenarios={
        # db variant : scenario
        'vanilla4' : 'vanilla4',
        'baseline' : 'baseline',
        "baseline_life_3" : "baseline_life_3",
        "baseline_life_7"   : "baseline_life_7",
    },
    scenario_labels={
        'vanilla4' : 'Vanilla',
        'baseline'  : 'Baseline',
        'baseline_life_3'  : '3 lifetime classes',
        'baseline_life_7'  : '7 lifetime classes',
    },
    modes_order=('LD Car', 'LD Truck', 'MD Truck', 'HD Truck')
)

In [49]:
def plot_capacity_flow_multicat_area_twocap(
        plot_title='Vehicle fleet stock in Ontario by vehicle class, powertrain and scenario',
        capacity_type='new capacity',
        cap_type_2='retired cap',  # e.g. 'new capacity' or 'retired cap'
        scenarios={
        # db variant : scenario
        'vanilla4' : 'vanilla4',
        'baseline' : 'baseline',
        "baseline_life_3" : "baseline_life_3",
        "baseline_life_7"   : "baseline_life_7",
        },
        scenario_labels={
            'vanilla4' : 'Vanilla',
            'baseline'  : 'Baseline',
            'baseline_life_3'  : '3 lifetime classes',
            'baseline_life_7'  : '7 lifetime classes',
        },
            modes_order=('LD Car', 'LD Truck', 'MD Truck', 'HD Truck'),
            color_map=color_fuel_map_rgba(0.9),
            color_map_2=color_fuel_map_rgba(0.6)
        ):

    # 1) Load filtered DataFrame:
    df = load_capacity_flow_scenarios(scenarios=scenarios, modes=modes_order)

    # 2) Build subplot titles (bold for top row):
    n_rows = len(modes_order)
    n_cols = len(tuple(scenarios.values()))
    subplot_titles = []
    for r, mode in enumerate(modes_order, start=1):
        for c, scen in enumerate(tuple(scenarios.values()), start=1):
            if r == 1:
                subplot_titles.append(f"<b>{scenario_labels.get(scen, scen)}</b>")
            else:
                subplot_titles.append("")

    # 3) Create figure with shared x & y axes:
    fig = make_subplots(
        rows=n_rows, cols=n_cols,
        shared_xaxes=True,
        shared_yaxes='rows',
        horizontal_spacing=0.015,
        vertical_spacing=0.03,
        subplot_titles=subplot_titles
    )

    # 4) Loop over each (mode, scenario) cell:
    for r_idx, mode in enumerate(modes_order, start=1):
        for c_idx, scen in enumerate(tuple(scenarios.values()), start=1):
            df_cell = df[(df['mode'] == mode) & (df['scenario'] == scen)].copy()
            if df_cell.empty:
                continue

            # 4a) Pivot primary capacity_type:
            pivot = (
                df_cell
                .groupby(['vintage', 'fuel'])[capacity_type]
                .sum()
                .unstack(fill_value=0)
            )
            pivot = pivot.reindex(columns=list(color_map.keys()), fill_value=0).sort_index()
            vintages = pivot.index.astype(str).tolist()

            # 4b) Add stacked‐area traces for capacity_type (stackgroup="one"):
            for i, fuel in enumerate(color_map.keys()):
                yvals = pivot[fuel].values.tolist()
                fill_mode = 'tozeroy' if i == 0 else 'tonexty'
                fig.add_trace(
                    go.Scatter(
                        x=vintages,
                        y=yvals,
                        mode='none',
                        name=fuel,
                        stackgroup="one",
                        fill=fill_mode,
                        fillcolor=color_map[fuel],
                        line=dict(color=color_map[fuel], width=0),
                        legendgroup=fuel,
                        showlegend=(r_idx == 1 and c_idx == 1),
                        hoverinfo='x+y+name'
                    ),
                    row=r_idx, col=c_idx
                )

            # 4c) If cap_type_2 is provided, pivot and add second stacked‐area (stackgroup="two"):
            if cap_type_2:
                pivot2 = (
                    df_cell
                    .groupby(['vintage', 'fuel'])[cap_type_2]
                    .sum()
                    .unstack(fill_value=0)
                )
                pivot2 = pivot2.reindex(columns=list(color_map.keys()), fill_value=0).sort_index()
                for i, fuel in enumerate(color_map.keys()):
                    yvals2 = pivot2[fuel].values.tolist()
                    fill_mode2 = 'tozeroy' if i == 0 else 'tonexty'
                    # Use slightly lower opacity for the second group so both are visible:
                    fig.add_trace(
                        go.Scatter(
                            x=vintages,
                            y=yvals2,
                            mode='none',
                            name=fuel,
                            stackgroup="two",
                            fill=fill_mode2,
                            fillcolor=color_map_2[fuel],
                            line=dict(color=color_map[fuel], width=0),
                            opacity=0.5,
                            legendgroup=fuel,
                            showlegend=False,
                            hoverinfo='x+y+name'
                        ),
                        row=r_idx, col=c_idx
                    )

    # 5) Tweak axes:
    # 5a) Primary y‐axis title only at (row=2, col=1):
    y_title = 'Fleet size (vehicles units)' if capacity_type == 'net capacity' else 'Capacity flow (vehicles units)'
    fig.update_yaxes(
        title_text=y_title,
        row=2, col=1
    )
    # 5b) Shared ticks & formatting:
    desired = 5                       # how many ticks you’d like
    for r_idx, mode in enumerate(modes_order, start=1):
        # grab all y-values from that row
        df_row = df[df["mode"] == mode]
        peak   = df_row[[capacity_type, cap_type_2]].abs().max().max()

        # pick a pleasant step (1–2–5 × 10^k)
        rough  = 2*peak / (desired-1)
        mag    = 10**np.floor(np.log10(rough))
        step   = min([1,2,5], key=lambda m: abs(rough - m*mag)) * mag

        fig.update_yaxes(
            range=[-peak, peak],   # symmetric about zero
            dtick=step,
            tickformat='~s',
            row=r_idx, col=1       # once per row is enough
        )
    # fig.update_yaxes(nticks=5, tickformat='~s', zeroline=True, zerolinecolor='black')
    # 5c) X‐axis: hide all except bottom row:
    fig.for_each_xaxis(lambda axis: axis.update(title_text='', showticklabels=False))
    fig.update_xaxes(
        showticklabels=True,
        tickangle=45,
        # ticklabelstandoff=2,
        row=n_rows, col=1
    )

    # 6) Layout and legend:
    fig.update_layout(
        width=220 * n_cols,
        height=200 * n_rows,
        margin=dict(t=70, b=100, l=20, r=20),
        title=dict(
            text=f"<b>{plot_title}</b>",
            x=0.5, y=0.985,
            xanchor='center', yanchor='top'
        ),
        yaxis_title_standoff=1,
        legend_title=dict(text="<b>Fuel/powertrain type</b>", font=dict(size=16)),
        legend_title_side='top right',
        barmode='stack',
        bargap=0.15,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.24,
            xanchor='center',
            x=0.5,
            traceorder='normal'
        ),
        font=dict(size=15),
        template='plotly_white'
    )

    # 7) Add bold row labels on the right of each row:
    for r_idx, mode in enumerate(modes_order, start=1):
        y_pos = 1 - (r_idx - 0.5) / n_rows
        fig.add_annotation(
            text=f"<b>{mode}</b>",
            x=1.005, xref='paper',
            y=y_pos, yref='paper',
            showarrow=False,
            textangle=90,
            xanchor='left', yanchor='middle',
            font=dict(size=16)
        )

    fig.show()
    fig.write_image("fleet_flows_lifes.svg", engine="kaleido", 
                    # scale=0.5
                    )

In [50]:
plot_capacity_flow_multicat_area_twocap(
    plot_title='Vehicle new capacity in Ontario by vehicle class, powertrain and scenario',
    capacity_type='new capacity',
    cap_type_2='retired cap',
    scenarios={
        # db variant : scenario
        'vanilla4' : 'vanilla4',
        'baseline'  : 'baseline',
        'baseline_life_3' : 'baseline_life_3',
        'baseline_life_7' : 'baseline_life_7',
        # 'embodied_life_7' : 'embodied_life_7',
        'medgrowth_life_7' : 'medgrowth_life_7',
        # 'embodied_medgrowth_life_7' : 'embodied_medgrowth_life_7',
    },
    scenario_labels={
        'vanilla4' : 'Vanilla',
        'baseline'  : 'Baseline',
        'baseline_life_3'  : '3 lifetime classes',
        'baseline_life_7'  : '7 lifetime classes',
        # 'embodied_life_7' : 'Embodied (7 life)',
        'medgrowth_life_7' : 'Med. growth (7 life)',
        # 'embodied_medgrowth_life_7' : 'Emb-Med. growth (7 life)',
    },
    modes_order=('LD Car', 'LD Truck', 'MD Truck', 'HD Truck')
)

In [27]:
plot_capacity_flow_multicat_area_twocap(
    plot_title='Vehicle new capacity in Ontario by vehicle class, powertrain and scenario',
    capacity_type='new capacity',
    cap_type_2='retired cap',
    scenarios={
        # db variant : scenario
        'vanilla4' : 'vanilla4',
        'baseline'  : 'baseline',
        'baseline_tailpipe' : 'baseline_tailpipe',
        'embodied' : 'embodied',
        'medgrowth' : 'medgrowth',
        'medgrowth_aeo' : 'medgrowth_aeo',
    },
    scenario_labels={
        'vanilla4' : 'Vanilla',
        'baseline'  : 'Baseline',
        'baseline_tailpipe' : 'Baseline (tailpipe)',
        'embodied' : 'Embodied',
        'medgrowth' : 'Med. growth',
        'medgrowth_aeo' : 'Med. growth (AEO)',
    },
    modes_order=('LD Car', 'LD Truck', 'MD Truck', 'HD Truck')
)